# Time Series Model Validation Demo

## Import Libraries 

In [1]:
# System libraries
import glob

# ML libraries
import pickle 
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import coint
from arch.unitroot import PhillipsPerron, DFGLS
import xgboost as xgb
from numpy import argmax
from sklearn.metrics import accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split

# Plotting libraries 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Load Time Series Models

In [2]:
with open('../models/time_series/fred_loan_rates_model_1.pkl', 'rb') as f:
    model_1 = pickle.load(f)
print(model_1.summary())

                                 OLS Regression Results                                
Dep. Variable:           MORTGAGE30US   R-squared (uncentered):                   0.286
Model:                            OLS   Adj. R-squared (uncentered):              0.284
Method:                 Least Squares   F-statistic:                              198.8
Date:                Tue, 09 May 2023   Prob (F-statistic):                    3.22e-38
Time:                        10:18:10   Log-Likelihood:                         -57.220
No. Observations:                 498   AIC:                                      116.4
Df Residuals:                     497   BIC:                                      120.7
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [3]:
with open('../models/time_series/fred_loan_rates_model_2.pkl', 'rb') as f:
    model_2 = pickle.load(f)
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:           MORTGAGE30US   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                  0.284
Method:                 Least Squares   F-statistic:                     198.3
Date:                Tue, 09 May 2023   Prob (F-statistic):           3.99e-38
Time:                        10:22:20   Log-Likelihood:                -57.120
No. Observations:                 498   AIC:                             118.2
Df Residuals:                     496   BIC:                             126.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0055      0.012     -0.448      0.6

In [4]:
with open('../models/time_series/fred_loan_rates_model_3.pkl', 'rb') as f:
    model_3 = pickle.load(f)
print(model_3.summary())

                                 OLS Regression Results                                
Dep. Variable:           MORTGAGE30US   R-squared (uncentered):                   0.529
Model:                            OLS   Adj. R-squared (uncentered):              0.528
Method:                 Least Squares   F-statistic:                              558.1
Date:                Tue, 09 May 2023   Prob (F-statistic):                    2.80e-83
Time:                        10:22:46   Log-Likelihood:                          46.439
No. Observations:                 498   AIC:                                     -90.88
Df Residuals:                     497   BIC:                                     -86.67
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [5]:
with open('../models/time_series/fred_loan_rates_model_4.pkl', 'rb') as f:
    model_4 = pickle.load(f)
print(model_4.summary())

                                 OLS Regression Results                                
Dep. Variable:           MORTGAGE30US   R-squared (uncentered):                   0.621
Model:                            OLS   Adj. R-squared (uncentered):              0.620
Method:                 Least Squares   F-statistic:                              407.0
Date:                Tue, 09 May 2023   Prob (F-statistic):                   2.45e-105
Time:                        10:23:06   Log-Likelihood:                          100.84
No. Observations:                 498   AIC:                                     -197.7
Df Residuals:                     496   BIC:                                     -189.3
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [6]:
with open('../models/time_series/fred_loan_rates_model_5.pkl', 'rb') as f:
    model_5 = pickle.load(f)
print(model_5.summary())

                                 OLS Regression Results                                
Dep. Variable:           MORTGAGE30US   R-squared (uncentered):                   0.622
Model:                            OLS   Adj. R-squared (uncentered):              0.619
Method:                 Least Squares   F-statistic:                              271.0
Date:                Tue, 09 May 2023   Prob (F-statistic):                   4.91e-104
Time:                        10:23:22   Log-Likelihood:                          100.97
No. Observations:                 498   AIC:                                     -195.9
Df Residuals:                     495   BIC:                                     -183.3
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

## Load Training Datasets

In [16]:
# Extract the endogenous (target) variable from the model fit
train_df = pd.Series(model_1.model.endog, index=model_1.model.data.row_labels)
train_df = train_df.to_frame()
target_var_name = model_1.model.endog_names
train_df.columns = [target_var_name]

# Extract the exogenous (explanatory) variables from the model fit
exog_df = pd.DataFrame(model_1.model.exog, index=model_1.model.data.row_labels, columns=model_1.model.exog_names)

# Concatenate the endogenous (target) and exogenous (explanatory) variables
train_df = pd.concat([train_df, exog_df], axis=1)
train_df.head()

,MORTGAGE30US,FEDFUNDS
DATE,,
2012-06-01,-0.09,0.00
2012-07-01,-0.17,0.00
2012-08-01,0.10,-0.03
2012-09-01,-0.19,0.01
2012-10-01,0.01,0.02


In [17]:
train_df.tail()

,MORTGAGE30US,FEDFUNDS
DATE,,
2012-06-01,-0.09,0.00
2012-07-01,-0.17,0.00
2012-08-01,0.10,-0.03
2012-09-01,-0.19,0.01
2012-10-01,0.01,0.02


## Load Test Datasets

In [15]:
file = '../datasets/time_series/fred_loan_rates_test_1.csv'
test_df = pd.read_csv(file, parse_dates=['DATE'], index_col='DATE')
display(test_df)

,MORTGAGE30US,UNRATE,GS10,FEDFUNDS
DATE,,,,
2012-11-01,3.32,7.7,1.65,0.16
2012-12-01,3.35,7.9,1.72,0.16
2013-01-01,3.53,8.0,1.91,0.14
2013-02-01,3.51,7.7,1.98,0.15
2013-03-01,3.57,7.5,1.96,0.14
...,...,...,...,...
2022-11-01,6.58,3.6,3.89,3.78
2022-12-01,6.42,3.5,3.62,4.10
2023-01-01,6.13,3.4,3.53,4.33


## Load Predictions

In [53]:
def get_model_prediction(model_fits, df_test):
    # Extract the training data from the first model fit
    train_data = pd.Series(model_fits[0].model.endog, index=model_fits[0].model.data.row_labels)
    train_data = train_data.to_frame()
    target_var_name = model_fits[0].model.endog_names
    train_data.columns = [f'{target_var_name}_train']

    # Initialize an empty DataFrame to store the predictions
    prediction_df = pd.DataFrame(index=df_test.index)
    prediction_df[f'{target_var_name}_test'] = np.nan

    # Concatenate the train_data and prediction_df
    combined_df = pd.concat([train_data, prediction_df], axis=0)

    # Loop through each model fit
    for i, model_fit in enumerate(model_fits):
        # Prepare the test dataset
        exog_names = model_fit.model.exog_names
        X_test = df_test.copy()

        # Add the constant if it's missing
        if 'const' in exog_names and 'const' not in X_test.columns:
            X_test['const'] = 1.0

        # Select the necessary columns
        X_test = X_test[exog_names]

        # Generate the predictions
        predictions = model_fit.predict(X_test)

        # Add the predictions to the DataFrame
        combined_df[f'model_{i+1}'] = np.nan
        combined_df[f'model_{i+1}'].iloc[len(train_data):] = predictions

    # Add the test data to the '<target_variable>_test' column
    combined_df[f'{target_var_name}_test'].iloc[len(train_data):] = df_test[target_var_name]

    return combined_df


In [54]:
model_fits = [model_1, model_2, model_3]  # Replace with your list of model fits
prediction_df = get_model_prediction(model_fits, test_df)
display(prediction_df)

,MORTGAGE30US_train,MORTGAGE30US_test,model_1,model_2,model_3
DATE,,,,,
1971-05-01,0.17,NaN,NaN,NaN,NaN
1971-06-01,0.08,NaN,NaN,NaN,NaN
1971-07-01,0.15,NaN,NaN,NaN,NaN
1971-08-01,0.00,NaN,NaN,NaN,NaN
1971-09-01,-0.02,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2022-11-01,NaN,6.58,1.096399,1.090463,2.889649
2022-12-01,NaN,6.42,1.189216,1.183240,2.689082
2023-01-01,NaN,6.13,1.255928,1.249923,2.622226
